In [8]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
# opening the downloaded shakespeare file
with open('input.txt', 'r') as f:
  text = f.read()
print(len(text))

1115393


In [10]:
print(text[:1000]) # These are some of the characters

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [11]:
chars = sorted(list(set(text)))
print(''.join(chars), len(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [12]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos  = {i:ch for ch,i in stoi.items()}
encode = lambda s : [stoi[ch] for ch in s]
decode = lambda l : [itos[i] for i in l]

In [13]:
# Now encoding the entire dataset in the given format
import torch
data = torch.tensor(encode(text), dtype = torch.long )
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [14]:
# Splitting into training and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[:n]

In [15]:
block_size = 8
train_data[:block_size+1] # We are doing this since we are preducting the next character for all the given indivivdual characters

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [16]:
x_train = train_data[:block_size]
y_train = train_data[1:block_size+1]
for t in range(block_size):
  context = x_train[:t+1]
  target = y_train[t]
  print(f'when input is {context} target is {target}')

when input is tensor([18]) target is 47
when input is tensor([18, 47]) target is 56
when input is tensor([18, 47, 56]) target is 57
when input is tensor([18, 47, 56, 57]) target is 58
when input is tensor([18, 47, 56, 57, 58]) target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [17]:
# setting a seed here
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print(xb.shape)

torch.Size([4, 8])


In [18]:
for ind_x, ind_y in zip(xb, yb):
  for t in range(block_size):
    context = ind_x[:t+1]
    target = ind_y[t]
    print(f'when input is {context} target is {target}')


when input is tensor([53]) target is 59
when input is tensor([53, 59]) target is 6
when input is tensor([53, 59,  6]) target is 1
when input is tensor([53, 59,  6,  1]) target is 58
when input is tensor([53, 59,  6,  1, 58]) target is 56
when input is tensor([53, 59,  6,  1, 58, 56]) target is 47
when input is tensor([53, 59,  6,  1, 58, 56, 47]) target is 40
when input is tensor([53, 59,  6,  1, 58, 56, 47, 40]) target is 59
when input is tensor([49]) target is 43
when input is tensor([49, 43]) target is 43
when input is tensor([49, 43, 43]) target is 54
when input is tensor([49, 43, 43, 54]) target is 1
when input is tensor([49, 43, 43, 54,  1]) target is 47
when input is tensor([49, 43, 43, 54,  1, 47]) target is 58
when input is tensor([49, 43, 43, 54,  1, 47, 58]) target is 1
when input is tensor([49, 43, 43, 54,  1, 47, 58,  1]) target is 58
when input is tensor([13]) target is 52
when input is tensor([13, 52]) target is 45
when input is tensor([13, 52, 45]) target is 43
when inp

In [19]:
# Making a bigram Language Model here
import torch.nn as nn
from torch.nn import functional as F

vocab_size = 65

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding = nn.Embedding(vocab_size,vocab_size) # here we are just using the count table to actually predict the next guy

  def forward(self, idx, targets = None):
    # Logits and targets are both B,T
    logits = self.token_embedding(idx) # B, T, C
    # Making it compatible to input into the below function
    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T,-1)
      targets = targets.view(-1)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss= self(idx)
      logits =  logits[:,-1,:] # (B, C)
      probs = F.softmax(logits, dim = -1)  # along (B,C)
      idx_next = torch.multinomial(probs, num_samples = 1)
      idx = torch.cat((idx, idx_next),dim=1)
      print(idx.shape)
    return idx
bigram = BigramLanguageModel(vocab_size)
logits, loss = bigram(xb, yb)

In [20]:
print(xb.shape)

torch.Size([4, 8])


In [21]:
n = BigramLanguageModel(vocab_size)
targets = torch.zeros(vocab_size)
# torch.zeros((1,1)logits.shape

In [22]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(loss, logits)


tensor(4.4065, grad_fn=<NllLossBackward0>) tensor([[-0.8588,  0.3182,  0.8569,  ...,  0.0516, -1.5103,  0.1050],
        [-2.0197, -0.6719,  0.9182,  ...,  0.2620, -2.0113,  0.2051],
        [-0.3555, -0.4067,  2.2399,  ...,  0.2392,  1.0608,  0.1914],
        ...,
        [-2.0197, -0.6719,  0.9182,  ...,  0.2620, -2.0113,  0.2051],
        [-0.7268,  2.0389, -0.3749,  ...,  1.0879, -0.0104, -0.9097],
        [ 0.3451,  0.4943, -0.3435,  ...,  0.1659, -0.9840, -0.0468]],
       grad_fn=<ViewBackward0>)


In [23]:
decode((m.generate(torch.zeros((1,1), dtype = torch.long),100))[0].tolist())
print('hello')

torch.Size([1, 2])
torch.Size([1, 3])
torch.Size([1, 4])
torch.Size([1, 5])
torch.Size([1, 6])
torch.Size([1, 7])
torch.Size([1, 8])
torch.Size([1, 9])
torch.Size([1, 10])
torch.Size([1, 11])
torch.Size([1, 12])
torch.Size([1, 13])
torch.Size([1, 14])
torch.Size([1, 15])
torch.Size([1, 16])
torch.Size([1, 17])
torch.Size([1, 18])
torch.Size([1, 19])
torch.Size([1, 20])
torch.Size([1, 21])
torch.Size([1, 22])
torch.Size([1, 23])
torch.Size([1, 24])
torch.Size([1, 25])
torch.Size([1, 26])
torch.Size([1, 27])
torch.Size([1, 28])
torch.Size([1, 29])
torch.Size([1, 30])
torch.Size([1, 31])
torch.Size([1, 32])
torch.Size([1, 33])
torch.Size([1, 34])
torch.Size([1, 35])
torch.Size([1, 36])
torch.Size([1, 37])
torch.Size([1, 38])
torch.Size([1, 39])
torch.Size([1, 40])
torch.Size([1, 41])
torch.Size([1, 42])
torch.Size([1, 43])
torch.Size([1, 44])
torch.Size([1, 45])
torch.Size([1, 46])
torch.Size([1, 47])
torch.Size([1, 48])
torch.Size([1, 49])
torch.Size([1, 50])
torch.Size([1, 51])
torch.Si

In [24]:
(m.generate(torch.zeros((1,1), dtype = torch.long),10)).shape

torch.Size([1, 2])
torch.Size([1, 3])
torch.Size([1, 4])
torch.Size([1, 5])
torch.Size([1, 6])
torch.Size([1, 7])
torch.Size([1, 8])
torch.Size([1, 9])
torch.Size([1, 10])
torch.Size([1, 11])


torch.Size([1, 11])

In [38]:
mau = torch.ones((3,3))
tril = torch.tril(torch.ones(3,3))
hello = mau.masked_fill_(tril==0, float('-inf'))

In [42]:
hello  =F.softmax(hello ,dim = 1)

In [638]:
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y



xb, yb = get_batch('train')
print(xb)

tensor([[59, 58, 47, 53, 52,  1, 61, 39],
        [52,  1, 51, 63,  1, 59, 52, 54],
        [ 0, 13, 26, 19, 17, 24, 27, 10],
        [43, 56, 57,  6,  1, 43, 56, 43]])


In [736]:
# making a single headed transformer
# Transforming into something understandable in the system and trying to make  a
block_size = 8
head = 8 
n_embd = 16
block_size = 8
class Head(nn.Module):
    def __init__(self,n_embd, head):
        super().__init__()
        self.keys = nn.Linear(n_embd, head, bias = False)
        self.queries = nn.Linear(n_embd, head, bias = False)
        self.values = nn.Linear(n_embd, head, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))


    def forward(self, x):
        B, T, C = x.shape
        K = self.keys(x)
        Q =  self.queries(x)
        V = self.values(x)
        wei = (K @ Q.transpose(-2, -1))*(C**(-0.5)) # B, T, C @ B, C, T  -->  B, T, T # Here C = Head 
        # print(wei)
        wei = wei.masked_fill_(self.tril[:T, :T] == 0, float('-inf')) # masking would mean the nodes are only communicating with nodes in the past and not the future ones 
        wei =  F.softmax(wei, -1)
        # print(wei)

        out = wei @ V # B,T,T
        # out = self.ln(out)
        # print(out.shape)
        
        return out 
    
# also joining a feedforward network in between


In [737]:
class MultiHead(nn.Module):
    def __init__(self, n_heads, head):
        super().__init__()
        self.heads = nn.ModuleList([Head(n_embd, head) for _ in range(n_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        print(out.shape)
        out = self.proj(out)
        return out 

In [738]:
# position embedding and token embedding
# token embedding  --> vocab_size to n_embd
# position_embedding --> 
head = 8
n_embd = 16
block_size = 8
# input  B,T format meh 
# what we want to find out 

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.position_embedding = nn.Embedding(block_size, n_embd)
        self.sa = nn.Sequential(MultiHead(4, head//4),
                      MultiHead(4, head//4),
                      MultiHead(4, head//4),
                     )

        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.linear = nn.Linear(n_embd, vocab_size)
        self.ffwd1 = nn.Linear(n_embd, n_embd*4) # we use all of this to make sure the model learns from the output of the 
        self.ffwd2 = nn.Linear(n_embd*4, n_embd)
        self.act1 = nn.ReLU()
        self.act2 = nn.ReLU()


    def forward(self, idx, targets = None):
        B, T = idx.shape
        token_embedding = self.token_embedding(idx)
        position_embedding  = self.position_embedding(torch.arange(T))
        x =  token_embedding  + position_embedding
        # print(x)
        # print(x)
        x = x +self.sa(x)
        x = self.ln1(x)   
        x = self.ffwd1(x)
        x = self.act1(x)
        x = self.ffwd2(x)
        x = self.act2(x)
        # print(x)
        # x = x + self.ln(out) # using resnet here, this is just adding the data as it is to  counter the vanishing gradient problem 
        # print(x)
        

        # print(x)

        # print(x)

        # print(x)
        # print(x)

        x = self.linear(x)

        
        if targets is None:
            return logits 
        else:
            B,T,C  = x.shape
            x = x.view(B*T, -1)
            targets = targets.view(-1)
            loss = F.cross_entropy(x, targets)
            return logits, loss
    
    

In [739]:
m = BigramLanguageModel()

In [740]:
import torch.optim as optim 

optimizer = optim.Adam(m.parameters(), lr = 1e-3)

In [741]:
# # making a MultiHead self attention 
# MultiHead attention usage
# x --> self.mh(x)  # No of Heads and the number of 
# class MultiHead(nn.Module):
#     self.heads = nn.ModuleList([for Head(head) for i in range()])

for i in range(10):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(f'loss for  step {i+1}: {loss}')

torch.Size([4, 8, 8])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x8 and 16x16)

In [515]:
x.type(float32)

NameError: name 'float32' is not defined

In [508]:
x = torch.tensor(32)
x.type(torch.float32)

tensor(32.)